# Import CSVs into OWID Grapher

The following notebook contains code for importing the UN SDG database into Grapher. It consumes the CSVs generated by a separate notebook.

### Notes

  - The length of the `name` column in the `datasets` table had to be changed to `512`: we have a `UNIQUE` constraint on it and the `namespace` column, and I was getting duplicates: `alter table datasets change name name varchar(512);`
  - We decided that this version of SDG needs to be imported into a new namespace (`un_sdg_2019`), as it was not possible to update the values of the existing version of SDG that is present in Grapher.

In [1]:
import pandas as pd
from tqdm import tqdm

from db import connection
from db_utils import DBUtils

from import_metadata import extract_description

pd.set_option('display.max_colwidth', -1)

# ID of user who imported the data
USER_ID = 29

# Dataset namespace
NAMESPACE = 'un_sdg_2019'

## Standardizing the entities

We use OpenRefine to standardize the entities. [**Read the instructions in the lc-reconcile repo**](https://github.com/owid/lc-reconcile) to perform the standardization on the `entities.csv` file.

At the end, you should have an `./entities_standardized.csv` file with columns:
- `id` (as the original CSV)
- `name` (as the original CSV)
- `standardized_name` (the database entity name)
- `db_entity_id` (the database entity id)

**There will likely be entities that were not matched with a database entity, those will be inserted below.**

In [15]:
entities = pd.read_csv('./entities_standardized.csv', index_col='id')

In [16]:
with connection as c:
    db = DBUtils(c)
    new_entities = entities[entities['db_entity_id'].isnull()]
    for _, entity in new_entities.iterrows():
        entity_id = entity.name
        entity_name = entity['name']
        db_entity_id = db.get_or_create_entity(entity_name)
        entities.loc[entity_id, 'db_entity_id'] = db_entity_id
        print(db_entity_id, entity['name'])

92682 Eastern Asia (excluding Japan and China)
92683 Iraq (Central Iraq)
92684 Iraq (Kurdistan Region)
92685 Western Asia (exc. Armenia, Azerbaijan, Cyprus, Israel and Georgia)
92686 Developed regions (Europe, Cyprus, Israel, Northern America, Japan, Australia & New Zealand)
92687 Eastern Asia (excluding Japan)
92688 Oceania (exc. Australia and New Zealand)
92689 Sub-Saharan Africa (inc. Sudan)
92690 Northern Africa (exc. Sudan)
92691 World Trade Organization (WTO) Member States
92692 WTO Developing Member States
92693 WTO Developed Member States
92694 International Centers (FAO)
92695 Regional Centres (FAO)


In [17]:
len(entities[entities['db_entity_id'].isnull()])

0

## Inserting the data

In [27]:
# [entity_id] → [db_entity_id] lookup
db_entity_id_by_entity_id = { row.name: int(row['db_entity_id']) for _, row in entities.iterrows() }

In [31]:
variables = pd.read_csv('./exported_data/variables.csv')
datasets = pd.read_csv('./exported_data/datasets.csv')
sources = pd.read_csv('./exported_data/sources.csv')

In [32]:
with connection as c:
    db = DBUtils(c)
    
    for _, dataset in datasets.iterrows():
        
        # Insert the dataset
        print("Inserting dataset: %s" % dataset['name'])
        db_dataset_id = db.upsert_dataset(name=dataset['name'], namespace=NAMESPACE, user_id=USER_ID)
        
        # Insert the source
        source = sources[sources['dataset_id'] == dataset.id].iloc[0]
        print("Inserting source: %s" % source['name'])
        db_source_id = db.upsert_source(name=source['name'], description=source['description'], dataset_id=db_dataset_id)
        
        # Insert variables associated with this dataset
        for j, variable in variables[variables.dataset_id == dataset['id']].iterrows():
            # insert row in variables table
            print("Inserting variable: %s" % variable['name'])
            db_variable_id = db.upsert_variable(
                name=variable['name'], 
                code=None, 
                unit=variable['unit'], 
                short_unit=None, 
                source_id=db_source_id, 
                dataset_id=db_dataset_id, 
                description=None)

            # read datapoints
            data_values = pd.read_csv('./exported_data/%04d_datapoints.csv' % variable.id)

            values = [(float(row['value']), int(row['year']), db_entity_id_by_entity_id[row['entity']], db_variable_id)
                      for _, row in data_values.iterrows()]

            print("Inserting values...")
            db.upsert_many("""
                INSERT INTO data_values (value, year, entityId, variableId)
                VALUES (%s, %s, %s, %s)
            """, values)
            print("Inserted %d values for variable" % len(values))


Inserting dataset: Proportion of population below international poverty line (%)
Inserting source: Proportion of population below international poverty line (%)
Inserting variable: 1.1.1 - Proportion of population below international poverty line (%) - SI_POV_DAY1
Inserting values...
Inserted 1822 values for variable
Inserting dataset: Employed population below international poverty line, by sex and age (%)
Inserting source: Employed population below international poverty line, by sex and age (%)
Inserting variable: 1.1.1 - Employed population below international poverty line, by sex and age (%) - SI_POV_EMP1 - 15 years old and over - Female
Inserting values...
Inserted 1019 values for variable
Inserting variable: 1.1.1 - Employed population below international poverty line, by sex and age (%) - SI_POV_EMP1 - 15 years old and over - Male
Inserting values...
Inserted 1019 values for variable
Inserting variable: 1.1.1 - Employed population below international poverty line, by sex and age

Inserted 339 values for variable
Inserting dataset: [World Bank] Poorest quintile covered by social assistance programs (%)
Inserting source: [World Bank] Poorest quintile covered by social assistance programs (%)
Inserting variable: 1.3.1 - [World Bank] Poorest quintile covered by social assistance programs (%) - SI_COV_SOCASTPQ
Inserting values...
Inserted 339 values for variable
Inserting dataset: [World Bank] Proportion of population covered by social insurance programs (%)
Inserting source: [World Bank] Proportion of population covered by social insurance programs (%)
Inserting variable: 1.3.1 - [World Bank] Proportion of population covered by social insurance programs (%) - SI_COV_SOCINS
Inserting values...
Inserted 322 values for variable
Inserting dataset: [World Bank] Poorest quintile covered by social insurance programs (%)
Inserting source: [World Bank] Poorest quintile covered by social insurance programs (%)
Inserting variable: 1.3.1 - [World Bank] Poorest quintile covered

Inserting values...
Inserted 223 values for variable
Inserting variable: 1.5.1 - Number damaged dwellings attributed to disasters, by hazard type (number) - VC_DSR_DDHN - Cold Wave
Inserting values...
Inserted 54 values for variable
Inserting variable: 1.5.1 - Number damaged dwellings attributed to disasters, by hazard type (number) - VC_DSR_DDHN - Avalanche
Inserting values...
Inserted 69 values for variable
Inserting variable: 1.5.1 - Number damaged dwellings attributed to disasters, by hazard type (number) - VC_DSR_DDHN - Snowstorm
Inserting values...
Inserted 113 values for variable
Inserting variable: 1.5.1 - Number damaged dwellings attributed to disasters, by hazard type (number) - VC_DSR_DDHN - Accident
Inserting values...
Inserted 293 values for variable
Inserting variable: 1.5.1 - Number damaged dwellings attributed to disasters, by hazard type (number) - VC_DSR_DDHN - Other
Inserting values...
Inserted 150 values for variable
Inserting variable: 1.5.1 - Number damaged dwelli

Inserted 54 values for variable
Inserting variable: 1.5.1 - Number destroyed dwellings attributed to disasters, by hazard type (number) - VC_DSR_DYHN - Earthquake
Inserting values...
Inserted 223 values for variable
Inserting variable: 1.5.1 - Number destroyed dwellings attributed to disasters, by hazard type (number) - VC_DSR_DYHN - Flood
Inserting values...
Inserted 562 values for variable
Inserting variable: 1.5.1 - Number destroyed dwellings attributed to disasters, by hazard type (number) - VC_DSR_DYHN - Windstorm
Inserting values...
Inserted 288 values for variable
Inserting variable: 1.5.1 - Number destroyed dwellings attributed to disasters, by hazard type (number) - VC_DSR_DYHN - Accident
Inserting values...
Inserted 293 values for variable
Inserting variable: 1.5.1 - Number destroyed dwellings attributed to disasters, by hazard type (number) - VC_DSR_DYHN - Hailstorm
Inserting values...
Inserted 172 values for variable
Inserting variable: 1.5.1 - Number destroyed dwellings at

Inserting variable: 1.5.1 - Number of injured or ill people attributed to disasters (number) - VC_DSR_IJILN
Inserting values...
Inserted 727 values for variable
Inserting dataset: Number of missing persons due to disaster (number)
Inserting source: Number of missing persons due to disaster (number)
Inserting variable: 1.5.1 - Number of missing persons due to disaster (number) - VC_DSR_MISS
Inserting values...
Inserted 343 values for variable
Inserting dataset: Number of deaths and missing persons attributed to disasters, by hazard type (number)
Inserting source: Number of deaths and missing persons attributed to disasters, by hazard type (number)
Inserting variable: 1.5.1 - Number of deaths and missing persons attributed to disasters, by hazard type (number) - VC_DSR_MMHN - Windstorm
Inserting values...
Inserted 288 values for variable
Inserting variable: 1.5.1 - Number of deaths and missing persons attributed to disasters, by hazard type (number) - VC_DSR_MMHN - Hailstorm
Inserting va

...

Inserting values...
Inserted 3129 values for variable
Inserting variable: 8.4.2 - Domestic material consumption, by type of raw material (tonnes) - EN_MAT_DOMCMPT - Crop residues
Inserting values...
Inserted 3357 values for variable
Inserting variable: 8.4.2 - Domestic material consumption, by type of raw material (tonnes) - EN_MAT_DOMCMPT - Oil shale and tar sands
Inserting values...
Inserted 54 values for variable
Inserting dataset: Average hourly earnings of managers (ISCO-08) (local currency)
Inserting source: Average hourly earnings of managers (ISCO-08) (local currency)
Inserting variable: 8.5.1 - Average hourly earnings of managers (ISCO-08) (local currency) - SL_EMP_AEARN - Female - All occupations (isco-08)
Inserting values...
Inserted 302 values for variable
Inserting variable: 8.5.1 - Average hourly earnings of managers (ISCO-08) (local currency) - SL_EMP_AEARN - Female - Service workers and shop and market sales workers (isco-88)
Inserting values...
Inserted 35 values for v

Inserting values...
Inserted 117 values for variable
Inserting variable: 8.5.1 - Average hourly earnings of managers (ISCO-08) (local currency) - SL_EMP_AEARN - Male - Craft and related trades workers (isco-08)
Inserting values...
Inserted 114 values for variable
Inserting variable: 8.5.1 - Average hourly earnings of managers (ISCO-08) (local currency) - SL_EMP_AEARN - Male - Not elsewhere classified (isco-88)
Inserting values...
Inserted 20 values for variable
Inserting variable: 8.5.1 - Average hourly earnings of managers (ISCO-08) (local currency) - SL_EMP_AEARN - Male - Not elsewhere classified (isco-08)
Inserting values...
Inserted 23 values for variable
Inserting variable: 8.5.1 - Average hourly earnings of managers (ISCO-08) (local currency) - SL_EMP_AEARN - Male - Armed forces occupations (isco-08)
Inserting values...
Inserted 33 values for variable
Inserting variable: 8.5.1 - Average hourly earnings of managers (ISCO-08) (local currency) - SL_EMP_AEARN - Male - Skilled agricul

Inserting source: Proportion of youth not in education, employment or training, by sex and age (%)
Inserting variable: 8.6.1 - Proportion of youth not in education, employment or training, by sex and age (%) - SL_TLF_NEET - 15 to 24 years old - Male
Inserting values...
Inserted 1322 values for variable
Inserting variable: 8.6.1 - Proportion of youth not in education, employment or training, by sex and age (%) - SL_TLF_NEET - 15 to 24 years old - Female
Inserting values...
Inserted 1322 values for variable
Inserting variable: 8.6.1 - Proportion of youth not in education, employment or training, by sex and age (%) - SL_TLF_NEET - 15 to 24 years old - Both sexes
Inserting values...
Inserted 1323 values for variable
Inserting variable: 8.6.1 - Proportion of youth not in education, employment or training, by sex and age (%) - SL_TLF_NEET - 15 to 29 years old - Male
Inserting values...
Inserted 2 values for variable
Inserting variable: 8.6.1 - Proportion of youth not in education, employment

Inserting values...
Inserted 902 values for variable
Inserting variable: 8.8.1 - Fatal occupational injuries among employees, by sex and migrant status (per 100,000 employees) - SL_EMP_FTLINJUR - No breakdown - Male
Inserting values...
Inserted 568 values for variable
Inserting variable: 8.8.1 - Fatal occupational injuries among employees, by sex and migrant status (per 100,000 employees) - SL_EMP_FTLINJUR - No breakdown - Female
Inserting values...
Inserted 562 values for variable
Inserting variable: 8.8.1 - Fatal occupational injuries among employees, by sex and migrant status (per 100,000 employees) - SL_EMP_FTLINJUR - Non-migrant - Both sexes
Inserting values...
Inserted 171 values for variable
Inserting variable: 8.8.1 - Fatal occupational injuries among employees, by sex and migrant status (per 100,000 employees) - SL_EMP_FTLINJUR - Non-migrant - Male
Inserting values...
Inserted 171 values for variable
Inserting variable: 8.8.1 - Fatal occupational injuries among employees, by s

Inserting variable: 9.1.2 - Freight volume, by mode of transport (tonne kilometres) - IS_RDP_FRGVOL - Air transport
Inserting values...
Inserted 38 values for variable
Inserting variable: 9.1.2 - Freight volume, by mode of transport (tonne kilometres) - IS_RDP_FRGVOL - Rail transport
Inserting values...
Inserted 285 values for variable
Inserting variable: 9.1.2 - Freight volume, by mode of transport (tonne kilometres) - IS_RDP_FRGVOL - Road transport
Inserting values...
Inserted 285 values for variable
Inserting dataset: Passenger volume (passenger kilometres), by mode of transport
Inserting source: Passenger volume (passenger kilometres), by mode of transport
Inserting variable: 9.1.2 - Passenger volume (passenger kilometres), by mode of transport - IS_RDP_PFVOL - Air transport
Inserting values...
Inserted 38 values for variable
Inserting variable: 9.1.2 - Passenger volume (passenger kilometres), by mode of transport - IS_RDP_PFVOL - Road transport
Inserting values...
Inserted 285 val

### Useful SQL for deleting all `data_values`, `variables`, `sources`, and `datasets` belonging to a `namespace`

```sql
DELETE data_values
FROM   data_values
       INNER JOIN variables
               ON variables.id = data_values.variableid
       INNER JOIN sources
               ON sources.id = variables.sourceid
       INNER JOIN datasets
               ON datasets.id = sources.datasetid
WHERE  datasets.namespace = 'un_sdg_2019';

DELETE variables
FROM   variables
       INNER JOIN sources
               ON sources.id = variables.sourceid
       INNER JOIN datasets
               ON datasets.id = sources.datasetid
WHERE  datasets.namespace = 'un_sdg_2019';

DELETE sources
FROM   sources
       INNER JOIN datasets
               ON datasets.id = sources.datasetid
WHERE  datasets.namespace = 'un_sdg_2019';

DELETE FROM datasets
WHERE  namespace = 'un_sdg_2019';  
```